In [1]:
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
from tqdm.notebook import tqdm as tqdm_notebook # tqdm – библиотека для визуализации прогресса в цикле
from collections import defaultdict
from statsmodels.stats.proportion import proportion_confint
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, palette='Set2')
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults', 
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

# Абсолютный критерий

In [2]:
def absolute_ttest(control, test):
    mean_control = np.mean(control)
    mean_test = np.mean(test)
    var_mean_control  = np.var(control) / len(control)
    var_mean_test  = np.var(test) / len(test)
    
    difference_mean = mean_test - mean_control
    difference_mean_var = var_mean_control + var_mean_test
    difference_distribution = sps.norm(loc=difference_mean, scale=np.sqrt(difference_mean_var))

    left_bound, right_bound = difference_distribution.ppf([0.025, 0.975])
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(difference_distribution.cdf(0), difference_distribution.sf(0))
    effect = difference_mean
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)



# 2. Создание тестируемого критерия
def cuped_ttest(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before
    return absolute_ttest(control_cup, test_cup)

AB-тест

In [3]:
# 3. Заводим счетчик
bad_cnt = 0

# 4. Цикл проверки
N = 30000
for i in tqdm_notebook(range(N)):
    # 4.a. Тестирую AB - тест
    control_before = sps.expon(scale=1000).rvs(1000)
    control = control_before + sps.norm(loc=0, scale=100).rvs(1000)

    test_before = sps.expon(scale=1000).rvs(1000)
    test = test_before + sps.norm(loc=0, scale=100).rvs(1000)
    test *= 1.1

    # 4.b. Запускаю критерий
    _, _, _, left_bound, right_bound = cuped_ttest(control, test, control_before, test_before)
    
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале
    if left_bound > 100 or right_bound < 100:
        bad_cnt += 1
        
# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

  0%|          | 0/30000 [00:00<?, ?it/s]

Реальный уровень значимости: 0.0504; доверительный интервал: [0.048, 0.0529]


## некорректная ковариата с "нулевым" средним

In [4]:
# 2. Создание тестируемого критерия
def incorrect_cuped(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))
    
    control_mean = np.mean(control_before)
    test_mean    = np.mean(test_before)

    control_cup = control - theta * (control_before - control_mean)
    test_cup = test - theta * (test_before - test_mean)
    return absolute_ttest(control_cup, test_cup)

AA-тест

In [5]:
# 4. Цикл проверки
N = 30000
for i in tqdm_notebook(range(N)):
    
    # 4.a. Тестирую AA - тест
    control_before = sps.expon(scale=1000).rvs(1000)
    control = control_before + sps.norm(loc=0, scale=100).rvs(1000)

    test_before = sps.expon(scale=1000).rvs(1000)
    test = test_before + sps.norm(loc=0, scale=100).rvs(1000)

    # 4.b. Запускаю критерий
    _, _, _, left_bound, right_bound = incorrect_cuped(control, test, control_before, test_before)
    
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале
    if left_bound > 0 or right_bound < 0:
        bad_cnt += 1
        
# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

  0%|          | 0/30000 [00:00<?, ?it/s]

Реальный уровень значимости: 0.8961; доверительный интервал: [0.8926, 0.8995]


# Относительный критерий


In [6]:
# 2. Создание тестируемого критерия
def relative_cuped(control, test, control_before, test_before):
    theta = (np.cov(control, control_before)[0, 1] + np.cov(test, test_before)[0, 1]) /\
                (np.var(control_before) + np.var(test_before))

    control_cup = control - theta * control_before
    test_cup = test - theta * test_before

    mean_den = np.mean(control)
    mean_num = np.mean(test_cup) - np.mean(control_cup)
    var_mean_den  = np.var(control) / len(control)
    var_mean_num  = np.var(test_cup) / len(test_cup) + np.var(control_cup) / len(control_cup)

    cov = -np.cov(control_cup, control)[0, 1] / len(control)

    relative_mu = mean_num / mean_den
    relative_var = var_mean_num / (mean_den ** 2)  + var_mean_den * ((mean_num ** 2) / (mean_den ** 4))\
                - 2 * (mean_num / (mean_den ** 3)) * cov
    
    relative_distribution = sps.norm(loc=relative_mu, scale=np.sqrt(relative_var))
    left_bound, right_bound = relative_distribution.ppf([0.025, 0.975])
    
    ci_length = (right_bound - left_bound)
    pvalue = 2 * min(relative_distribution.cdf(0), relative_distribution.sf(0))
    effect = relative_mu
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

AB-тест

In [7]:
# 3. Заводим счетчик
bad_cnt = 0

# 4. Цикл проверки
N = 30000
cis = []
for i in tqdm_notebook(range(N)):
    # 4.a. Тестирую AB - тест
    control_before = sps.expon(scale=1000).rvs(1000)
    control = control_before + sps.norm(loc=0, scale=100).rvs(1000)

    test_before = sps.expon(scale=1000).rvs(1000)
    test = test_before + sps.norm(loc=0, scale=100).rvs(1000)
    test *= 1.1

    # 4.b. Запускаю критерий
    _, _, ci, left_bound, right_bound = relative_cuped(control, test, control_before, test_before)
    cis.append(ci)
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале
    if left_bound > 0.1 or right_bound < 0.1:
        bad_cnt += 1

# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

  0%|          | 0/30000 [00:00<?, ?it/s]

Реальный уровень значимости: 0.0518; доверительный интервал: [0.0494, 0.0544]
